In [ ]:

import numpy as np
import cv2
from skimage.feature import hog
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline 



In [ ]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features



In [ ]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features




In [ ]:
# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [ ]:

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features


In [ ]:
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], xy_window=(64, 64), xy_overlap=(0.5, 0.5)):

        if x_start_stop[0] == None:
            x_start_stop[0] = 0
        if x_start_stop[1] == None:
            x_start_stop[1] = img.shape[1]
        if y_start_stop[0] == None:
            y_start_stop[0] = 0
        if y_start_stop[1] == None:
            y_start_stop[1] = img.shape[0]
        
        # Compute the span of the region to be searched    
        xspan = x_start_stop[1] - x_start_stop[0]
        yspan = y_start_stop[1] - y_start_stop[0]
        # Compute the number of pixels per step in x/y
        nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
        ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
        # Compute the number of windows in x/y
        nx_windows = np.int(xspan/nx_pix_per_step) - 1
        ny_windows = np.int(yspan/ny_pix_per_step) - 1
        # Initialize a list to append window positions to
        window_list = []
        # Loop through finding x and y window positions
        # Note: you could vectorize this step, but in practice
        # you'll be considering windows one by one with your
        # classifier, so looping makes sense
        for ys in range(ny_windows):
            for xs in range(nx_windows):
                # Calculate window position
                startx = xs*nx_pix_per_step + x_start_stop[0]
                endx = startx + xy_window[0]
                starty = ys*ny_pix_per_step + y_start_stop[0]
                endy = starty + xy_window[1]
                
                # Append window position to list
                window_list.append(((startx, starty), (endx, endy)))
        # Return the list of windows
        return window_list



In [ ]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


In [ ]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    #plt.imshow(feature_image)
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)
    
    #9) Return concatenated array of features
    return np.concatenate(img_features)




In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    counter = 0
    test_images_list = []
    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        counter += 1
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64,64))      
        test_images_list.append(test_img)
        #plt.imshow(test_img)
        
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        y_proba = clf.predict_proba(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
            
    #8) Return windows for positive detections
    return on_windows, test_images_list
    

In [ ]:
# Read in cars and notcars
images = glob.glob('train/*.jpeg')
cars = []
notcars = []
for image in images:
    image_name = image.split('/')
    if 'image' in image_name[1] or 'extra' in image_name[1] :
        notcars.append(image)
    else:
        image
        cars.append(image)

sample_size = 1000
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (64, 64) # Spatial binning dimensions
hist_bins = 64    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off


car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)


X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


In [ ]:
def car_classifier(X, y):
    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)

    print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))
    # Use a linear SVC 
    svc = LinearSVC(C=1.0)
    svc = CalibratedClassifierCV(svc) 
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    # Check the prediction time for a single sample
    t=time.time()
    
    return svc

In [ ]:
svc = car_classifier(scaled_X, y)

In [ ]:
def calculate_range_window(hot_window):
    x_list = []
    y_list = []
    hot_window_out = []
    for point1, point2 in hot_window:
        x_list.append(point1)
        y_list.append(point2)
    tolerance_x = (x_list[1]-x_list[0])
    tolerance_y = (y_list[1]-y_list[0])

    
    for point1, point2 in hot_window:
        pointx_max = point1 + tolerance_x
        pointy_max = point2 + tolerance_y
        pointx_min = point1 - tolerance_x
        pointy_min = point2 - tolerance_y
        hot_window_out.append ((((pointx_min),(pointy_min)), ((pointx_max),(pointy_max) )))
    
    return hot_window_out[0], hot_window_out[1]


In [ ]:
class Bbox():
    '''Will hndle Bounding Boxes'''
    
    def __init__(self):
        # was a bbox detected in the last frame?
        self.detected = False  
        
        # current position x_start_stop and y_start_stop
        self.list_position = [((),())]  
        
        #count for overlaping windows
        self.count = 0
        

        
        
class Car():
    
    def __init__(self, position, current_frame):
        # was a car detected in the last frame?
        self.position = position #((,),(,))
        self.frame = current_frame
        
    def is_this_the_same(self, hot_window):
        p1_range, p2_range =  calculate_range_window(self.position)

        x1 = p1_range[0][0]<= hot_window[0][0]<= p1_range[1][0]
        y1 = p1_range[0][1]<= hot_window[0][1]<= p1_range[1][1]

        x2 = p2_range[0][0]<= hot_window[1][0]<= p2_range[1][0]
        y2 = p2_range[0][1]<= hot_window[1][1]<= p2_range[1][1]
        
        if x1 and y1 and x2 and y2:
            return True
        
        else:
            return False
        
    def integrate_extra_window(self, hot_window, current_frame = 0):
        bb_x_start = hot_window[0][0]
        bb_y_start = hot_window[0][1]
        
        bb_x_end = hot_window[1][0]
        bb_y_end = hot_window[1][1]
        
        car_x_start = self.position[0][0]
        car_y_start = self.position[0][1]
        
        car_x_end = self.position[1][0]
        car_y_end = self.position[1][1]
        
        
        
        if current_frame == self.frame:
            
            if bb_x_start < car_x_start:
                car_x_start = bb_x_start
                
            if bb_y_start > car_y_start:
                car_y_start = bb_y_start
                
            if bb_x_end > car_x_end:
                car_x_end = bb_x_end
            if bb_y_end > car_x_end:
                car_x_end = bb_y_end
        
            self.position = ((car_x_start,car_y_start),(car_x_end,car_y_end))
            
        else:
            
            mean_x_start = int(np.mean(list([bb_x_start,car_x_start])))
            mean_y_start = int(np.mean(list([bb_y_start,car_y_start])))
            mean_x_end = int(np.mean(list([bb_x_end,car_x_end])))
            mean_y_end = int(np.mean(list([bb_y_end,car_y_end])))
        
            self.position = hot_window

        
        self.frame = current_frame
        return self.position 
        
        


In [ ]:
def pipeline(img, current_frame):
    global car_list
    new_hot_windows = []
    new_car_list = []
    windows_to_draw = []

    y_start_stop = [400, 700] # Min and max in y to search in slide_window()
    #windows_all= []
    draw_image = np.copy(img)
    image_size = draw_image.shape
    center_x = 700
    center_y = 440 
    
    windows_small = slide_window(draw_image, x_start_stop=[610, 1110], y_start_stop=[400, 490],
                                 xy_window=(90, 90), xy_overlap=(0.5, 0.5))
    windows_medium = slide_window(draw_image, x_start_stop=[530, 1280], y_start_stop=[380, 550], 
                                  xy_window=(150, 150), xy_overlap=(0.5, 0.5))

    windows_all =  windows_medium  + windows_small 

    hot_windows, x = search_windows(draw_image, windows_all, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)   
    
    if len(hot_windows)> 0:
        Bbox.detected = True
        hot_window_taken = False
        for hot_window in hot_windows:
            for car in car_list:
                if car.is_this_the_same(hot_window):
                    
                    new_hot_window = car.integrate_extra_window(hot_window,current_frame)
                    
                    hot_window_taken = True
                    break
                else:
                    hot_window_taken = False
            if not hot_window_taken:
                new_car = Car(hot_window, current_frame)
                car_list.append(new_car)
                
                
    counter_car = 0
    for car in car_list:
        
        if current_frame - car.frame <= 8: #5 #3 #
            new_hot_windows.append(car.position)
            new_car_list.append(car)
            
        else:            
            pass
            
    car_list = new_car_list
    
          
    for hot_window in new_hot_windows:
        for car in car_list:
            if car.is_this_the_same(hot_window):
                windows_to_draw.append(hot_window)
                
            else:
                pass
                    
            
        
        
    
        
    window_img = draw_boxes(draw_image, windows_to_draw, color=(255, 0, 0), thick=6)  
    return window_img

In [ ]:
car_list = []
counter = 0
def counter_frames(image):
    global counter
    img = image
    
    result = pipeline(img,counter )
    counter +=1
    
    return result



In [ ]:
#To process the video 
output = 'video_output.mp4'

file = VideoFileClip("project_video.mp4");

out = file.fl_image(counter_frames) #NOTE: this function expects color images!!
%time out.write_videofile(output, audio=False);